In [ ]:
!pip install transformers==4.18.0 
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import datasets
import warnings # to ignore warnings
from utils import squad_json_to_dataframe_train
warnings.filterwarnings('ignore')

In [ ]:
language = 'zh'
pretraining_data_path = f"./squad.pretraining.{language}.csv"

In [ ]:
from transformers import AutoTokenizer, LineByLineTextDataset

# model_name = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=pretraining_data_path, block_size=128)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, AutoModelForMaskedLM

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True, mlm_probability=0.15)    

 
training_args = TrainingArguments(
    output_dir="{language}-TAPT-MLM-MiniLM",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=8,
    save_total_limit=1,
    push_to_hub=True
)

In [ ]:
model_name = "subhasisj/MiniLMv2-qa-encoder"
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset)

trainer.train()

In [ ]:
trainer.push_to_hub('Training Completed')